# Importing Spark

In [ ]:
# install Java JDK 8
# install SPARK
# install pyspark
# eventually consider pyspark
# eventuelly consider SPARK_HOME, or SPARK_PATH, etc.

In [ ]:
import os
os.environ['JAVA_HOME'] = '/Library/Java/JavaVirtualMachines/adoptopenjdk-8.jdk/Contents/Home'

In [ ]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession

sc = SparkContext.getOrCreate()
spark = SparkSession.builder.appName("Python Spark").getOrCreate()

# Loading the data

In [ ]:
df_ratings = spark.read\
    .option("delimiter", "\t")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .csv('data/u.data')

In [ ]:
# print the dataframe schema
df_ratings.printSchema()

In [ ]:
# show a sample of the data (the dataframe executes the whole pipeline at this stage)
df_ratings.show(5)

In [ ]:
# set the rdd equivalent of the dataframe
rdd_ratings = df_ratings.rdd

# Basic Queries

#### Exercice 1 - Number of movies per user (using RDD then Dataframe)

Calculer pour chaque utilisateur le nombre de films notés, et afficher le résultat pour l'un d'entre eux. Utilisez dans un premier temps les RDD puis les Dataframes.

In [ ]:
# classical RDD approach
result_1 = ...

In [ ]:
# dataframe approach, filtering on the previous user to compare results
df_ratings....

#### Exercice 2 - Average rating per user (using RDD then Dataframe)

Calculer pour chaque utilisateurs la note moyenne donnée et afficher le résultat pour l'un d'entre eux. Utilisez dans un premier temps les RDD puis les Dataframes.

In [ ]:
# RDD

In [ ]:
# dataframe approach

#### Exercice 3 - Top-5 movies with at least 15 votes (Dataframe)

Afficher les 5 meilleurs films parmi ceux qui ont reçu au moins 15 votes.

*Indices:*
* Utiliser df_ratings pour calculer la moyenne, filtrer les films qui ont moins de 15 notes et classer les films par ordre décroissant.
* Faire un join avec df_movies pour afficher le nom des films sélectionnés.

In [ ]:
df_items = spark.read\
    .option("delimiter", "|")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .csv('data/u.item')